
## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/netflix_titles_sample.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)
df.show()

show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
s1,Movie,Inception,Christopher Nolan,Leonardo DiCaprio,USA,"September 1, 2020",2010,PG-13,148 min,"Sci-Fi & Fantasy, Action",A thief who steals corporate secrets...
s2,TV Show,Breaking Bad,Vince Gilligan,Bryan Cranston,USA,"January 1, 2021",2008,TV-MA,5 Seasons,"Crime TV Shows, Drama",A chemistry teacher turned meth kingpin...
s3,Movie,The Irishman,Martin Scorsese,Robert De Niro,USA,"November 27, 2019",2019,R,209 min,"Dramas, Crime",A mob hitman recalls his past...
s4,Movie,Bird Box,Susanne Bier,Sandra Bullock,USA,"December 14, 2018",2018,R,124 min,"Horror Movies, Thriller",A woman must protect herself...
s5,TV Show,Stranger Things,The Duffer Brothers,Millie Bobby Brown,USA,"July 15, 2016",2016,TV-14,3 Seasons,"TV Sci-Fi & Fantasy, Drama",A group of kids uncover a dark secret...


+-------+-------+---------------+-------------------+------------------+-------+-----------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|          title|           director|              cast|country|       date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+---------------+-------------------+------------------+-------+-----------------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|      Inception|  Christopher Nolan| Leonardo DiCaprio|    USA|September 1, 2020|        2010| PG-13|  148 min|Sci-Fi & Fantasy,...|A thief who steal...|
|     s2|TV Show|   Breaking Bad|     Vince Gilligan|    Bryan Cranston|    USA|  January 1, 2021|        2008| TV-MA|5 Seasons|Crime TV Shows, D...|A chemistry teach...|
|     s3|  Movie|   The Irishman|    Martin Scorsese|    Robert De Niro|    USA|November 27, 2019|        2019|     R|  209 min|       Dramas, Cr

In [0]:
# Create a view or table

temp_table_name = "netflix_titles_sample_csv"

df.createOrReplaceTempView(temp_table_name)

In [0]:
%sql

/* Query the created temp table in a SQL cell */

select * from `netflix_titles_sample_csv`

In [0]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "netflix_titles_sample_csv"

# df.write.format("parquet").saveAsTable(permanent_table_name)

In [0]:
df.groupBy("type").count().show()


+-------+-----+
|   type|count|
+-------+-----+
|TV Show|    2|
|  Movie|    3|
+-------+-----+



In [0]:
from pyspark.sql.functions import split, explode, trim

# Split listed_in into array of genres
df_genres = df.withColumn("genre", explode(split(df["listed_in"], ",")))

# Remove extra spaces
df_genres = df_genres.withColumn("genre", trim(df_genres["genre"]))

# Group and count genres
df_genres.groupBy("genre").count().orderBy("count", ascending=False).show()


+-------------------+-----+
|              genre|count|
+-------------------+-----+
|              Drama|    2|
|              Crime|    1|
|           Thriller|    1|
|   Sci-Fi & Fantasy|    1|
|     Crime TV Shows|    1|
|             Dramas|    1|
|      Horror Movies|    1|
|             Action|    1|
|TV Sci-Fi & Fantasy|    1|
+-------------------+-----+



In [0]:
df.groupBy("rating").count().orderBy("count", ascending=False).show()

+------+-----+
|rating|count|
+------+-----+
|     R|    2|
| TV-MA|    1|
| TV-14|    1|
| PG-13|    1|
+------+-----+



In [0]:
from pyspark.sql.functions import regexp_extract, col

# Filter only Movies
movies_df = df.filter(df["type"] == "Movie")

# Extract numeric duration from "148 min"
movies_df = movies_df.withColumn("duration_minutes", regexp_extract(col("duration"), r"(\d+)", 1).cast("int"))

# View results
movies_df.select("title", "duration", "duration_minutes").show()


+------------+--------+----------------+
|       title|duration|duration_minutes|
+------------+--------+----------------+
|   Inception| 148 min|             148|
|The Irishman| 209 min|             209|
|    Bird Box| 124 min|             124|
+------------+--------+----------------+



In [0]:
from pyspark.sql.functions import avg

movies_df.select(avg("duration_minutes").alias("avg_movie_duration")).show()


+------------------+
|avg_movie_duration|
+------------------+
|160.33333333333334|
+------------------+

